In [1]:
# Вспомогательные библиотеки
# Работа с датафреймом
import pandas as pd
# Линейная алгебра
import numpy as np
# Рисование графиков
import matplotlib.pyplot as plt
# Для логов
import datetime

In [2]:
# Tensorflow
import tensorflow as tf


In [3]:
# Keras
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.python.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard

In [4]:
# Настройки генератора случайных чисел
from tensorflow import random
random.set_seed(31)
np.random.seed(31)

In [5]:
# sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

### Подготовка данных

In [6]:
# Считываем датасеты
train = pd.read_csv('data/fashion-mnist_train.csv')
test = pd.read_csv('data/fashion-mnist_test.csv')

In [7]:
# train-это датасет на котором мы будем тренировать модели
# test - это датасет на котором мы будем проверять качество наших моделей
train.shape,test.shape

((60000, 785), (10000, 785))

In [17]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Редактируем датафреймы.
# Тренировочный
x_train = train.drop('label',axis=1)/255
y_train = train['label']
# Тестовый
x_test = test.drop('label',axis=1)/255
y_test = test['label']

In [20]:
# Кодируем категориальный целевой признак в 0 или 1
# Создаем экземляр класса
encoder = OneHotEncoder()

In [21]:
# Вид до преобразования
y_train.head()

0    2
1    9
2    6
3    0
4    3
Name: label, dtype: int64

In [14]:
y_train_labels = encoder.fit_transform(np.reshape(np.array(y_train), (-1,1))).toarray()
y_test_labels = encoder.transform(np.reshape(np.array(y_test), (-1,1))).toarray()

In [22]:
# Вид после
y_train_labels[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

###  Логистическая регрессия


In [56]:
# Разделим тренировочные данные(x_train и y_train) еще раз на тренировочную
# и валидационную.
x_TRAIN, x_VALID, y_TRAIN, y_VALID = train_test_split(
    x_train,y_train_labels,test_size=0.2,
random_state = 31)

In [57]:
# Создаем модель
logistick_model = Sequential()
# Добавляем в нее полносвязный слой 
logistick_model.add(Dense(10, input_shape=(784,), activation='softmax'))

In [58]:
# Настраиваем процесс обучения
logistick_model.compile(loss='categorical_crossentropy',
                        optimizer='sgd',
                        metrics=['accuracy'])

In [59]:
# Создаем callbacks
# Остановка обучения если результат перестал улучшаться
early_stopping =  EarlyStopping(patience=3, monitor='val_loss')
# Tensorboard
# profile_batch чтобы не выскакивала ошибка профилировщика
tensorboard = TensorBoard(log_dir=f'Logs\\{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}',
                          profile_batch = 100000000)
callbacks = [early_stopping,tensorboard]

In [36]:
# # Загружаем веса модели из файла.
# logistick_model.load_weights('Logistic_model_stohastic_gradient.h5')
# # Проверяем на тестовых данных
# _, score = logistick_model.evaluate(x_test.values, y_test_labels)
# print("Логистическая регрессия   accuracy: {0:.3f}".format(score))


In [60]:
# # Обучаем модель
# logistick_model.fit(x_TRAIN.values, y_TRAIN, batch_size=500,
#                     validation_data=(x_VALID.values, y_VALID),
#                        callbacks=callbacks, epochs=1000)

Train on 48000 samples, validate on 12000 samples
Epoch 1/1000
48000/48000 [==============================] - 4s 93us/sample - loss: 1.7520 - accuracy: 0.4412 - val_loss: 1.3652 - val_accuracy: 0.6088
Epoch 2/1000
48000/48000 [==============================] - 1s 29us/sample - loss: 1.2069 - accuracy: 0.6479 - val_loss: 1.0987 - val_accuracy: 0.6664
Epoch 3/1000
48000/48000 [==============================] - 1s 25us/sample - loss: 1.0257 - accuracy: 0.6870 - val_loss: 0.9767 - val_accuracy: 0.6910
Epoch 4/1000
48000/48000 [==============================] - 1s 22us/sample - loss: 0.9317 - accuracy: 0.7080 - val_loss: 0.9050 - val_accuracy: 0.7137
Epoch 5/1000
48000/48000 [==============================] - 2s 32us/sample - loss: 0.8721 - accuracy: 0.7244 - val_loss: 0.8554 - val_accuracy: 0.7252
Epoch 6/1000
48000/48000 [==============================] - 1s 25us/sample - loss: 0.8296 - accuracy: 0.7361 - val_loss: 0.8189 - val_accuracy: 0.7364
Epoch 7/1000
48000/48000 [==================

48000/48000 [==============================] - 1s 12us/sample - loss: 0.5326 - accuracy: 0.8247 - val_loss: 0.5425 - val_accuracy: 0.8223
Epoch 55/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.5312 - accuracy: 0.8252 - val_loss: 0.5414 - val_accuracy: 0.8229
Epoch 56/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.5297 - accuracy: 0.8258 - val_loss: 0.5398 - val_accuracy: 0.8227
Epoch 57/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.5283 - accuracy: 0.8257 - val_loss: 0.5392 - val_accuracy: 0.8238
Epoch 58/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.5270 - accuracy: 0.8264 - val_loss: 0.5372 - val_accuracy: 0.8244
Epoch 59/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.5256 - accuracy: 0.8265 - val_loss: 0.5361 - val_accuracy: 0.8247
Epoch 60/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.5243 - accuracy:

Epoch 108/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4842 - accuracy: 0.8378 - val_loss: 0.4970 - val_accuracy: 0.8324
Epoch 109/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4836 - accuracy: 0.8380 - val_loss: 0.4973 - val_accuracy: 0.8338
Epoch 110/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4831 - accuracy: 0.8377 - val_loss: 0.4960 - val_accuracy: 0.8342
Epoch 111/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4826 - accuracy: 0.8381 - val_loss: 0.4958 - val_accuracy: 0.8345
Epoch 112/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4821 - accuracy: 0.8380 - val_loss: 0.4955 - val_accuracy: 0.8334
Epoch 113/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4815 - accuracy: 0.8383 - val_loss: 0.4945 - val_accuracy: 0.8334
Epoch 114/1000
48000/48000 [==============================] - 1s 13us/sample - los

48000/48000 [==============================] - 1s 13us/sample - loss: 0.4619 - accuracy: 0.8429 - val_loss: 0.4766 - val_accuracy: 0.8412
Epoch 162/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4616 - accuracy: 0.8434 - val_loss: 0.4768 - val_accuracy: 0.8403
Epoch 163/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4613 - accuracy: 0.8431 - val_loss: 0.4760 - val_accuracy: 0.8415
Epoch 164/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4609 - accuracy: 0.8434 - val_loss: 0.4759 - val_accuracy: 0.8412
Epoch 165/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4606 - accuracy: 0.8434 - val_loss: 0.4755 - val_accuracy: 0.8408
Epoch 166/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4603 - accuracy: 0.8434 - val_loss: 0.4753 - val_accuracy: 0.8413
Epoch 167/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4600 - acc

48000/48000 [==============================] - 1s 13us/sample - loss: 0.4479 - accuracy: 0.8468 - val_loss: 0.4647 - val_accuracy: 0.8441
Epoch 215/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4477 - accuracy: 0.8468 - val_loss: 0.4640 - val_accuracy: 0.8439
Epoch 216/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4475 - accuracy: 0.8470 - val_loss: 0.4639 - val_accuracy: 0.8438
Epoch 217/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4474 - accuracy: 0.8470 - val_loss: 0.4636 - val_accuracy: 0.8439
Epoch 218/1000
48000/48000 [==============================] - 1s 15us/sample - loss: 0.4471 - accuracy: 0.8471 - val_loss: 0.4639 - val_accuracy: 0.8433
Epoch 219/1000
48000/48000 [==============================] - 1s 31us/sample - loss: 0.4469 - accuracy: 0.8469 - val_loss: 0.4640 - val_accuracy: 0.8447
Epoch 220/1000
48000/48000 [==============================] - 1s 18us/sample - loss: 0.4467 - acc

48000/48000 [==============================] - 1s 14us/sample - loss: 0.4382 - accuracy: 0.8504 - val_loss: 0.4560 - val_accuracy: 0.8464
Epoch 268/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4381 - accuracy: 0.8505 - val_loss: 0.4558 - val_accuracy: 0.8462
Epoch 269/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4378 - accuracy: 0.8506 - val_loss: 0.4554 - val_accuracy: 0.8462
Epoch 270/1000
48000/48000 [==============================] - 1s 16us/sample - loss: 0.4378 - accuracy: 0.8506 - val_loss: 0.4558 - val_accuracy: 0.8468
Epoch 271/1000
48000/48000 [==============================] - 1s 14us/sample - loss: 0.4376 - accuracy: 0.8503 - val_loss: 0.4553 - val_accuracy: 0.8466
Epoch 272/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4374 - accuracy: 0.8507 - val_loss: 0.4551 - val_accuracy: 0.8465
Epoch 273/1000
48000/48000 [==============================] - 1s 13us/sample - loss: 0.4373 - acc

In [61]:
# # # Сохраняем  полученные веса модели
# logistick_model.save_weights('Logistic_model_stohastic_gradient.h5',
#                              save_format ='h5')

### График качества модели(метрика accuracy) на тренировочной и  валидационной выборках в зависимости от количества эпох.


<img src ='data\logistick_model_accuracy.jpg'>

### Проверяем качество модели на тестовой выборке

In [69]:
_, score = logistick_model.evaluate(x_test.values, y_test_labels)
print("Логистическая регрессия \nAccuracy на тестовой выборке равна: {0:.4f}".format(score))

10000/10000 [==============================] - 0s 48us/sample - loss: 0.4510 - accuracy: 0.8503
Логистическая регрессия 
Accuracy на тестовой выборке равна: 0.8503


Наша модель показывает похожие результаты как валидационной выборке, так и на тестовой. Что может говорить о том что она не переобучилась на тренировочных данных.